In [1]:
import json
import os
import numpy as np

In [2]:
def import_metrics_json(models_path, run_num):
    """
    Looks in {models_path}/{run_num}/metrics.json and returns the contents as a
    Python dictionary. Returns None if the path does not exist.
    """
    path = os.path.join(models_path, str(run_num), "metrics.json")
    if not os.path.exists(path):
        return None
    with open(path, "r") as f:
        return json.load(f)

In [3]:
def get_best_metric(models_path, metric_name, reduce_func, compare_func):
    """
    Given the path to a set of runs, determines the run with the best metric value,
    for the given `metric_name`. For each run, the function `reduce_func` must take
    the array of all values for that metric and return a (scalar) value to use for
    comparison. The best metric value is determined by `metric_compare_func`, which
    must take in two arguments, and return whether or not the _first_ one is better.
    Returns the number of the run, the value associated with that run, and a dict of
    all the values used for comparison.
    """
    # Get the metrics, ignoring empty or nonexistent metrics.json files
    metrics = {run_num : import_metrics_json(models_path, run_num) for run_num in os.listdir(models_path)}
    metrics = {key : val for key, val in metrics.items() if val}  # Remove empties
    
    # Get the best value
    best_run, best_val, all_vals = None, None, {}
    for run_num in metrics.keys():
        try:
            val = reduce_func(metrics[run_num][metric_name]["values"])
            all_vals[run_num] = val
            if best_val is None or compare_func(val, best_val):
                best_val, best_run = val, run_num
        except Exception:
            print("Warning: Was not able to compute values for run %s" % run_num)
            continue
    return best_run, best_val, all_vals

In [4]:
def get_best_metric_at_best_epoch(models_path, metric_name, reduce_func, compare_func):
    """
    Given the path to a set of runs, determines the run with the best metric value,
    for the given `metric_name`. For each run, the function `reduce_func` must take
    the array of all values for that metric and return a (scalar) value FOR EACH
    SUBARRAY/VALUE in the value array to use for comparison. The best metric value
    is determined by `metric_compare_func`, which must take in two arguments, and
    return whether or not the _first_ one is better.
    Returns the number of the run, the (one-indexed) number of the epch, the value
    associated with that run and epoch, and a dict of all the values used for
    comparison (mapping pair of run number and epoch number to value).
    """
    # Get the metrics, ignoring empty or nonexistent metrics.json files
    metrics = {run_num : import_metrics_json(models_path, run_num) for run_num in os.listdir(models_path)}
    metrics = {key : val for key, val in metrics.items() if val}  # Remove empties
    
    # Get the best value
    best_run, best_epoch, best_val, all_vals = None, None, None, {}
    for run_num in metrics.keys():
        try:
            # Find the best epoch within that run
            best_epoch_in_run, best_val_in_run = None, None
            for i, subarr in enumerate(metrics[run_num][metric_name]["values"]):
                val = reduce_func(subarr)
                if best_val_in_run is None or compare_func(val, best_val_in_run):
                    best_epoch_in_run, best_val_in_run = i + 1, val
            all_vals[(run_num, best_epoch_in_run)] = best_val_in_run
            
            # If the best value in the best epoch of the run is best so far, update
            if best_val is None or compare_func(best_val_in_run, best_val):
                best_run, best_epoch, best_val = run_num, best_epoch_in_run, best_val_in_run
        except Exception:
            print("Warning: Was not able to compute values for run %s" % run_num)
            continue
    return best_run, best_epoch, best_val, all_vals

In [9]:
models_path = "/users/amtseng/att_priors/models/trained_models/profile_models/SPI1/"
best_run, best_epoch, best_val, all_vals = get_best_metric_at_best_epoch(
    models_path,
    "val_prof_corr_losses",
    lambda values: np.mean(values),
    lambda x, y: x < y
)
print("Best run: %s" % best_run)
print("Best epoch in run: %d" % best_epoch)
print("Associated value: %s" % best_val)
for key in sorted(all_vals.keys(), key=lambda p: int(p[0])):
    print(key, all_vals[key])

Best run: 2
Best epoch in run: 5
Associated value: 88.49771720484684
('1', 4) 88.78697790710712
('2', 5) 88.49771720484684
('3', 5) 89.79084527069976
('4', 3) 89.58271916570776
('5', 9) 89.26148537739263
('6', 4) 88.99320590944218
('7', 10) 89.59271423798755
('8', 10) 91.59313578462395
('9', 10) 91.41927015116286
('10', 9) 91.56127111023146
('11', 4) 89.8861976590791
('12', 4) 89.50611362539136
('13', 4) 89.67429481146162
('14', 5) 89.1128744049564
('16', 7) 88.88628069832602
('17', 9) 90.23008309271103
('18', 5) 89.66416141989407
('19', 5) 88.89251274043328
('20', 9) 89.35180024250494
('21', 6) 90.39302847211454
('22', 6) 88.93682920740703
('24', 10) 88.89174473643814
('25', 10) 89.76119854165967
('26', 10) 89.36809022436837


In [8]:
for key in sorted(all_vals.keys(), key=lambda p: int(p[0])):
    print(key[0])
    metrics = import_metrics_json(models_path, key[0])
    print(np.mean(metrics["val_prof_corr_losses"]["values"], axis=1))
    print(np.mean(metrics["val_pos_att_losses"]["values"], axis=1))

1
[90.19145586 90.34649422 90.67525145 89.97432134 90.21670427]
[0.18087088 0.18326705 0.18710049 0.17815944 0.16360778]
2
[92.24133602 90.58685371 90.04016878 89.56698248 89.40778269]
[0.21906283 0.18467457 0.16983209 0.14657481 0.12994495]
3
[90.1000459  89.87048591 89.64201466 89.86513184 89.78246824]
[0.1651428  0.10460002 0.08746735 0.07634063 0.07132788]
4
[90.62560846 89.57861705 89.41195201 89.3727135  89.49493291]
[0.16301606 0.12651851 0.09835131 0.08491801 0.07552006]
5
[107.88571157 102.59444964  99.63890778  97.31083254  96.08157368]
[0.55474781 0.54745899 0.38743676 0.28041846 0.23903734]
6
[89.99119638 89.4621466  89.37868028 89.13201996 89.81746649]
[0.16768231 0.17261086 0.17185755 0.16561546 0.1543162 ]
7
[91.37146831 89.97360968 89.58680592 89.50709701 89.48905542]
[0.18007919 0.11724876 0.08574675 0.07304158 0.06286086]
8
[90.82598595 90.89919637 90.58726524 90.58332741 90.32673512]
[0.1514177  0.16285462 0.16404581 0.17734207 0.16189879]
9
[90.66459173 89.88057034 